In [72]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.sports_betting_capstone
nfl_lines = db.nfl_lines
nfl_results = db.nfl_results

In [143]:
nfl_lines_row = nfl_lines.find()
nfl_lines_pddf = pd.DataFrame(list(nfl_lines_row))
nfl_lines_pddf

,_id,Team,Win-Loss,Streak,Open,Closing,Consensus_Spread,Consensus_Money,Consensus_OU,Season,Week
0,5f98877a1e3e51b1b60328e4,451 Houston,(0-0) (0-0 A),,56.5,53.5,50%,26%,72%,2020,1
1,5f98877a1e3e51b1b60328e5,452 Kansas City Â«,(0-0) (0-0 H),,-10,-9.5,50%,74%,28%,2020,1
2,5f98877a1e3e51b1b60328e6,0 Las Vegas Â«,(0-0) (0-0 A),,,,,,,2020,1
3,5f98877a1e3e51b1b60328e7,0 Carolina,(0-0) (0-0 H),,,,,,,2020,1
4,5f98877a1e3e51b1b60328e8,453 Miami,(0-0) (0-0 A),,43.5,41.5,42%,51%,58%,2020,1
...,...,...,...,...,...,...,...,...,...,...,...
2339,5f98883cc01cbbbbde303746,314 New England Â«,(14-2) (6-2 H),W-8,,-5.5,55%,40%,34%,2016,20
2340,5f98883dc01cbbbbde303748,497 AFC,(0-0) (0-0 A),,68.5,80,34%,69%,77%,2016,21
2341,5f98883dc01cbbbbde303749,498 NFC,(0-0) (0-0 H),,-4.5,-4,66%,31%,23%,2016,21
2342,5f98883ec01cbbbbde30374b,101 New England,(14-2) (8-0 A),W-9,PK,-3,63%,47%,52%,2016,22


In [144]:
for idx, row in nfl_lines_pddf.iterrows():
    if idx == 0 or idx % 2 == 0:
        nfl_lines_pddf.loc[idx, 'Include'] = True
        nfl_lines_pddf.loc[idx, 'Team 1'] = nfl_lines_pddf.loc[idx, 'Team']
        nfl_lines_pddf.loc[idx, 'Team 2'] = nfl_lines_pddf.loc[idx + 1, 'Team']
        nfl_lines_pddf.loc[idx, 'Spread Open'] = nfl_lines_pddf.loc[idx , 'Open']
        nfl_lines_pddf.loc[idx, 'Spread Close'] = nfl_lines_pddf.loc[idx, 'Closing']
        nfl_lines_pddf.loc[idx, 'Spread Consensus Team 1'] = nfl_lines_pddf.loc[idx, 'Consensus_Spread']
        nfl_lines_pddf.loc[idx, 'Spread Consensus Team 2'] = nfl_lines_pddf.loc[idx + 1, 'Consensus_Spread']
        nfl_lines_pddf.loc[idx, 'O/U Open'] = nfl_lines_pddf.loc[idx + 1 , 'Open']
        nfl_lines_pddf.loc[idx, 'O/U Close'] = nfl_lines_pddf.loc[idx + 1, 'Closing']
        nfl_lines_pddf.loc[idx, 'Over Consensus'] = nfl_lines_pddf.loc[idx, 'Consensus_OU']
        nfl_lines_pddf.loc[idx, 'Under Consensus'] = nfl_lines_pddf.loc[idx + 1, 'Consensus_OU']
        nfl_lines_pddf.loc[idx, 'Moneyline Consensus Team 1'] = nfl_lines_pddf.loc[idx, 'Consensus_Money']
        nfl_lines_pddf.loc[idx, 'Moneyline Consensus Team 2'] = nfl_lines_pddf.loc[idx + 1, 'Consensus_Money']
    else:
        nfl_lines_pddf.loc[idx, 'Include'] = False

In [145]:
clean_nfl_lines_pddf = nfl_lines_pddf.drop(nfl_lines_pddf.columns[range(9)], axis = 1)[nfl_lines_pddf['Include']].reset_index()

In [146]:
clean_nfl_lines_pddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172 entries, 0 to 1171
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   index                       1172 non-null   int64 
 1   Season                      1172 non-null   int64 
 2   Week                        1172 non-null   int64 
 3   Include                     1172 non-null   object
 4   Team 1                      1172 non-null   object
 5   Team 2                      1172 non-null   object
 6   Spread Open                 1172 non-null   object
 7   Spread Close                1172 non-null   object
 8   Spread Consensus Team 1     1172 non-null   object
 9   Spread Consensus Team 2     1172 non-null   object
 10  O/U Open                    1172 non-null   object
 11  O/U Close                   1172 non-null   object
 12  Over Consensus              1172 non-null   object
 13  Under Consensus             1172 non-null   obje

In [140]:
clean_nfl_lines_pddf

,index,Season,Week,Include,Team 1,Team 2,Spread Open,Spread Close,Spread Consensus Team 1,Spread Consensus Team 2,O/U Open,O/U Close,Over Consensus,Under Consensus,Moneyline Consensus Team 1,Moneyline Consensus Team 2
0,0,2020,1,True,451 Houston,452 Kansas City Â«,56.5,53.5,50%,50%,-10,-9.5,72%,28%,26%,74%
1,2,2020,1,True,0 Las Vegas Â«,0 Carolina,,,,,,,,,,
2,4,2020,1,True,453 Miami,454 New England Â«,43.5,41.5,42%,58%,-5.5,-7,58%,42%,51%,49%
3,6,2020,1,True,455 Cleveland,456 Baltimore Â«,48.5,47.5,40%,60%,-9,-7,72%,28%,25%,75%
4,8,2020,1,True,457 N.Y. Jets,458 Buffalo Â«,40.0,39,20%,80%,-5.5,-6,49%,51%,23%,77%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,2334,2016,19,True,307 Green Bay Â«,308 Dallas,51.5,53.5,56%,44%,-4,-5,76%,24%,77%,23%
1168,2336,2016,20,True,311 Green Bay,312 Atlanta Â«,61,61,65%,35%,-4,-6,63%,37%,72%,28%
1169,2338,2016,20,True,313 Pittsburgh,314 New England Â«,,49.5,45%,55%,,-5.5,66%,34%,60%,40%
1170,2340,2016,21,True,497 AFC,498 NFC,68.5,80,34%,66%,-4.5,-4,77%,23%,69%,31%


In [147]:
clean_nfl_lines_pddf['Team 1'] = clean_nfl_lines_pddf['Team 1'].apply(lambda x: ''.join(filter(str.isalpha, x))).str.replace('Â',"")
clean_nfl_lines_pddf['Team 2'] = clean_nfl_lines_pddf['Team 2'].apply(lambda x: ''.join(filter(str.isalpha, x))).str.replace('Â',"")

In [148]:
clean_nfl_lines_pddf

,index,Season,Week,Include,Team 1,Team 2,Spread Open,Spread Close,Spread Consensus Team 1,Spread Consensus Team 2,O/U Open,O/U Close,Over Consensus,Under Consensus,Moneyline Consensus Team 1,Moneyline Consensus Team 2
0,0,2020,1,True,Houston,KansasCity,56.5,53.5,50%,50%,-10,-9.5,72%,28%,26%,74%
1,2,2020,1,True,LasVegas,Carolina,,,,,,,,,,
2,4,2020,1,True,Miami,NewEngland,43.5,41.5,42%,58%,-5.5,-7,58%,42%,51%,49%
3,6,2020,1,True,Cleveland,Baltimore,48.5,47.5,40%,60%,-9,-7,72%,28%,25%,75%
4,8,2020,1,True,NYJets,Buffalo,40.0,39,20%,80%,-5.5,-6,49%,51%,23%,77%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,2334,2016,19,True,GreenBay,Dallas,51.5,53.5,56%,44%,-4,-5,76%,24%,77%,23%
1168,2336,2016,20,True,GreenBay,Atlanta,61,61,65%,35%,-4,-6,63%,37%,72%,28%
1169,2338,2016,20,True,Pittsburgh,NewEngland,,49.5,45%,55%,,-5.5,66%,34%,60%,40%
1170,2340,2016,21,True,AFC,NFC,68.5,80,34%,66%,-4.5,-4,77%,23%,69%,31%


In [73]:
nfl_results = nfl_results.find()
nfl_results_pddf = pd.DataFrame(list(nfl_results))
nfl_results_pddf

,_id,Team,Final,Season,Week
0,5f993e2bd6f3f1cd63a4cc57,Oakland (12-4),14,2016,18
1,5f993e2bd6f3f1cd63a4cc58,Houston (9-7),27,2016,18
2,5f993e2bd6f3f1cd63a4cc59,Detroit (9-7),6,2016,18
3,5f993e2bd6f3f1cd63a4cc5a,Seattle (10-5-1),26,2016,18
4,5f993e2bd6f3f1cd63a4cc5b,Miami (10-6),12,2016,18
...,...,...,...,...,...
2341,5f993f6e0a7014855df1ea2c,LA Chargers (2-4),39,2020,7
2342,5f993f6e0a7014855df1ea2d,Seattle (5-1),34,2020,7
2343,5f993f6e0a7014855df1ea2e,Arizona (5-2),37,2020,7
2344,5f993f6e0a7014855df1ea2f,Chicago (5-2),10,2020,7


In [95]:
for idx, row in nfl_results_pddf.iterrows():
    if idx == 0 or idx % 2 == 0:
        nfl_results_pddf.loc[idx, 'Include'] = True
        nfl_results_pddf.loc[idx, 'Team 1'] = nfl_results_pddf.loc[idx, 'Team']
        nfl_results_pddf.loc[idx, 'Team 2'] = nfl_results_pddf.loc[idx + 1, 'Team']
        nfl_results_pddf.loc[idx, 'Team 1 Final'] = int(nfl_results_pddf.loc[idx , 'Final'])
        nfl_results_pddf.loc[idx, 'Team 2 Final'] = int(nfl_results_pddf.loc[idx + 1, 'Final'])
        nfl_results_pddf.loc[idx, 'Total Final'] = int(nfl_results_pddf.loc[idx, 'Final']) + int(nfl_results_pddf.loc[idx + 1, 'Final'])
    else:
        nfl_results_pddf.loc[idx, 'Include'] = False

In [96]:
clean_nfl_results_pddf = nfl_results_pddf.drop(['_id', 'Team', 'Final'], axis = 1)[nfl_results_pddf['Include']].reset_index()

In [97]:
clean_nfl_results_pddf

,index,Season,Week,Include,Team 1,Team 2,Team 1 Final,Team 2 Final,Total Final
0,0,2016,18,True,Oakland (12-4),Houston (9-7),14,27,41
1,2,2016,18,True,Detroit (9-7),Seattle (10-5-1),6,26,32
2,4,2016,18,True,Miami (10-6),Pittsburgh (11-5),12,30,42
3,6,2016,18,True,NY Giants (11-5),Green Bay (10-6),13,38,51
4,8,2016,19,True,Seattle (10-5-1),Atlanta (11-5),20,36,56
...,...,...,...,...,...,...,...,...,...
1168,2336,2020,7,True,Kansas City (6-1),Denver (2-4),43,16,59
1169,2338,2020,7,True,San Francisco (4-3),New England (2-4),33,6,39
1170,2340,2020,7,True,Jacksonville (1-6),LA Chargers (2-4),29,39,68
1171,2342,2020,7,True,Seattle (5-1),Arizona (5-2),34,37,71


In [98]:
clean_nfl_results_pddf['Team 1'] = clean_nfl_results_pddf['Team 1'].apply(lambda x: ''.join(filter(str.isalpha, x)))
clean_nfl_results_pddf['Team 2'] = clean_nfl_results_pddf['Team 2'].apply(lambda x: ''.join(filter(str.isalpha, x)))

In [118]:
clean_nfl_results_pddf['Season'] = clean_nfl_results_pddf['Season'].astype(str)
clean_nfl_results_pddf['Week'] = clean_nfl_results_pddf['Week'].astype(str)
clean_nfl_results_pddf['Game ID 1'] = clean_nfl_results_pddf['Season'] + clean_nfl_results_pddf['Week'] + clean_nfl_results_pddf['Team 1'] + clean_nfl_results_pddf['Team 2']
clean_nfl_results_pddf['Game ID 2'] = clean_nfl_results_pddf['Season'] + clean_nfl_results_pddf['Week'] + clean_nfl_results_pddf['Team 2'] + clean_nfl_results_pddf['Team 1']

In [125]:
clean_nfl_results_pddf

,index,Season,Week,Include,Team 1,Team 2,Team 1 Final,Team 2 Final,Total Final,Game ID 1,Game ID 2
0,0,2016,18,True,Oakland,Houston,14,27,41,201618OaklandHouston,201618HoustonOakland
1,2,2016,18,True,Detroit,Seattle,6,26,32,201618DetroitSeattle,201618SeattleDetroit
2,4,2016,18,True,Miami,Pittsburgh,12,30,42,201618MiamiPittsburgh,201618PittsburghMiami
3,6,2016,18,True,NYGiants,GreenBay,13,38,51,201618NYGiantsGreenBay,201618GreenBayNYGiants
4,8,2016,19,True,Seattle,Atlanta,20,36,56,201619SeattleAtlanta,201619AtlantaSeattle
...,...,...,...,...,...,...,...,...,...,...,...
1168,2336,2020,7,True,KansasCity,Denver,43,16,59,20207KansasCityDenver,20207DenverKansasCity
1169,2338,2020,7,True,SanFrancisco,NewEngland,33,6,39,20207SanFranciscoNewEngland,20207NewEnglandSanFrancisco
1170,2340,2020,7,True,Jacksonville,LAChargers,29,39,68,20207JacksonvilleLAChargers,20207LAChargersJacksonville
1171,2342,2020,7,True,Seattle,Arizona,34,37,71,20207SeattleArizona,20207ArizonaSeattle


In [149]:
clean_nfl_lines_pddf['Season'] = clean_nfl_lines_pddf['Season'].astype(str)
clean_nfl_lines_pddf['Week'] = clean_nfl_lines_pddf['Week'].astype(str)
clean_nfl_lines_pddf['Game ID 1'] = clean_nfl_lines_pddf['Season'] + clean_nfl_lines_pddf['Week'] + clean_nfl_lines_pddf['Team 1'] + clean_nfl_lines_pddf['Team 2']
clean_nfl_lines_pddf['Game ID 2'] = clean_nfl_lines_pddf['Season'] + clean_nfl_lines_pddf['Week'] + clean_nfl_lines_pddf['Team 2'] + clean_nfl_lines_pddf['Team 1']

In [150]:
clean_nfl_lines_pddf

,index,Season,Week,Include,Team 1,Team 2,Spread Open,Spread Close,Spread Consensus Team 1,Spread Consensus Team 2,O/U Open,O/U Close,Over Consensus,Under Consensus,Moneyline Consensus Team 1,Moneyline Consensus Team 2,Game ID 1,Game ID 2
0,0,2020,1,True,Houston,KansasCity,56.5,53.5,50%,50%,-10,-9.5,72%,28%,26%,74%,20201HoustonKansasCity,20201KansasCityHouston
1,2,2020,1,True,LasVegas,Carolina,,,,,,,,,,,20201LasVegasCarolina,20201CarolinaLasVegas
2,4,2020,1,True,Miami,NewEngland,43.5,41.5,42%,58%,-5.5,-7,58%,42%,51%,49%,20201MiamiNewEngland,20201NewEnglandMiami
3,6,2020,1,True,Cleveland,Baltimore,48.5,47.5,40%,60%,-9,-7,72%,28%,25%,75%,20201ClevelandBaltimore,20201BaltimoreCleveland
4,8,2020,1,True,NYJets,Buffalo,40.0,39,20%,80%,-5.5,-6,49%,51%,23%,77%,20201NYJetsBuffalo,20201BuffaloNYJets
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,2334,2016,19,True,GreenBay,Dallas,51.5,53.5,56%,44%,-4,-5,76%,24%,77%,23%,201619GreenBayDallas,201619DallasGreenBay
1168,2336,2016,20,True,GreenBay,Atlanta,61,61,65%,35%,-4,-6,63%,37%,72%,28%,201620GreenBayAtlanta,201620AtlantaGreenBay
1169,2338,2016,20,True,Pittsburgh,NewEngland,,49.5,45%,55%,,-5.5,66%,34%,60%,40%,201620PittsburghNewEngland,201620NewEnglandPittsburgh
1170,2340,2016,21,True,AFC,NFC,68.5,80,34%,66%,-4.5,-4,77%,23%,69%,31%,201621AFCNFC,201621NFCAFC


In [152]:
clean_nfl_lines_pddf.to_csv('clean_nfl_lines_output.csv')
clean_nfl_results_pddf.to_csv('clean_nfl_results_output.csv')

In [153]:
combined_nfl_pddf = pd.merge(clean_nfl_lines_pddf, clean_nfl_results_pddf, left_on = ['Game ID 1', 'Game ID 2'], right_on = ['Game ID 1', 'Game ID 2'])

In [154]:
combined_nfl_pddf

,index_x,Season_x,Week_x,Include_x,Team 1_x,Team 2_x,Spread Open,Spread Close,Spread Consensus Team 1,Spread Consensus Team 2,...,Game ID 2,index_y,Season_y,Week_y,Include_y,Team 1_y,Team 2_y,Team 1 Final,Team 2 Final,Total Final
0,0,2020,1,True,Houston,KansasCity,56.5,53.5,50%,50%,...,20201KansasCityHouston,2136,2020,1,True,Houston,KansasCity,20,34,54
1,2,2020,1,True,LasVegas,Carolina,,,,,...,20201CarolinaLasVegas,2144,2020,1,True,LasVegas,Carolina,34,30,64
2,10,2020,1,True,LasVegas,Carolina,PK,-3,55%,45%,...,20201CarolinaLasVegas,2144,2020,1,True,LasVegas,Carolina,34,30,64
3,4,2020,1,True,Miami,NewEngland,43.5,41.5,42%,58%,...,20201NewEnglandMiami,2152,2020,1,True,Miami,NewEngland,11,21,32
4,6,2020,1,True,Cleveland,Baltimore,48.5,47.5,40%,60%,...,20201BaltimoreCleveland,2140,2020,1,True,Cleveland,Baltimore,6,38,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,2330,2016,19,True,Houston,NewEngland,45,44.5,32%,68%,...,201619NewEnglandHouston,10,2016,19,True,Houston,NewEngland,16,34,50
1150,2332,2016,19,True,Pittsburgh,KansasCity,PK,45.5,56%,44%,...,201619KansasCityPittsburgh,14,2016,19,True,Pittsburgh,KansasCity,18,16,34
1151,2334,2016,19,True,GreenBay,Dallas,51.5,53.5,56%,44%,...,201619DallasGreenBay,12,2016,19,True,GreenBay,Dallas,34,31,65
1152,2336,2016,20,True,GreenBay,Atlanta,61,61,65%,35%,...,201620AtlantaGreenBay,16,2016,20,True,GreenBay,Atlanta,21,44,65


In [155]:
combined_nfl_pddf.to_csv('combined_output.csv')